We want to use a foreloop to load the data and pass it to a delta table  
Infer schema, header true


In [0]:
dbutils.fs.head("/Volumes/bike_data_lakehouse/raw_data/source_crm/cust_info.csv")

declare Catalog
write with raw_data schema  
for loop the volume  
for loop table  
read table  
save in bronze schema

In [0]:
import sys
from pyspark.sql import DataFrame
from typing import Any
sys.path.append('/Workspace/databricks_bootcamp/bronze/')

In [0]:
from meta_driven_ingestion import INGESTION
catalog = "bike_data_lakehouse"
ingestion_schema = "raw_data"
output_schema = "bronze"


TO DO: Create table with schema

In [0]:
def missing_configs(config_dict: dict) -> None:
    REQUIRED_FIELDS: list[str] = ["source", "table_type", "allow_overwrite"]

    missing_configs:list[str] = [field for field in REQUIRED_FIELDS if field not in config_dict]

    if missing_configs:
        raise ValueError(f"Missing required metadata fields: {missing_configs}")

    return None

def overwrite_allowed(original_table_name: str, config_dict: dict) -> None:
    # Check overwrite logic is allwoed
    if not config_dict.get("allow_overwrite"):
        raise ValueError(f"{original_table_name} does not allow table overwrite")

    return None

def valid_file_type(original_table_name: str, config_dict: dict)-> None:
    supported_types = {'CSV'} #has to be in upper
    if config_dict['table_type'].upper() not in supported_types:
        raise ValueError(f"ERROR: Unsupported file {original_table_name}")

    return None

def path_checker(path: str)-> bool:
    # Check if path exists in 
    try:
        dbutils.fs.ls(path)
        return True
    except Exception:
        return False
    
def empty_table(row_count: int, original_table_name: str)-> None:
    # Check if table is empty
    if row_count < 1:
        raise ValueError(f"Data frame is empty: {original_table_name}")

In [0]:
def ingestion(meta_data:dict[str, Any]) -> None:

    for volume, table_dicts in meta_data.items():
        for clean_table_name, config_dict in table_dicts.items():
            # Metadata validation
            missing_configs(config_dict= config_dict)

            original_table_name: str = config_dict["source"]

            # Check overwrite logic is allwoed
            overwrite_allowed(original_table_name=original_table_name, config_dict=config_dict)
            
            # Check table for valid file type
            valid_file_type(original_table_name=original_table_name, config_dict=config_dict)

            print(f"Ingesting {original_table_name}")

            reader = spark.read.option("header", "true")
            path: str = f"/Volumes/{catalog}/{ingestion_schema}/{volume}/{original_table_name}.csv"
            
            # Check if path exists
            if not path_checker(path):
                raise ValueError(f"File not found: {path}")
            
            df:DataFrame = reader.csv(path)

            row_count:int = df.count()

            # Table is empty
            empty_table(row_count=row_count, original_table_name=original_table_name)
            
            print(f"number of rows in {clean_table_name}: {row_count}")

            data_frame_columns = df.columns
            print(f"columns of table {original_table_name}: {data_frame_columns}")

            df.write.mode("overwrite").format("delta").saveAsTable(f"{catalog}.{output_schema}.{clean_table_name}")

In [0]:
ingestion(meta_data=INGESTION)

{'source_crm': {'customer_info': {'source': 'cust_info', 'table_type': 'CSV'}, 'product_info': {'source': 'prd_info', 'table_type': 'CSV'}, 'sales_info': {'source': 'sales_details', 'table_type': 'CSV'}}, 'source_erp': {'Customer_AZ_12': {'source': 'CUST_AZ12', 'table_type': 'CSV'}, 'Location_A101': {'source': 'LOC_A101', 'table_type': 'CSV'}, 'Product_Category_G1_V2': {'source': 'PX_CAT_G1V2', 'table_type': 'CSV'}}}